<a href="https://colab.research.google.com/github/GitMishka/reddit_api-to_db/blob/master/watch_exchange_to_db1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [193]:
!pip install praw
import praw
from google.colab import drive
import pandas as pd
import datetime
import psycopg2 as ps


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [194]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

In [195]:
# #database credentials
# host_name = 'my-db1.cwc1bi2tp4j7.us-east-2.rds.amazonaws.com'
# dbname = 'initial_db1'
# port = '5432'
# username = 'postgres' 
# password = 'Manonthemoon123'
# conn = None

# #establish a connection to db
# conn = connect_to_db(host_name, dbname, port, username, password)
# curr = conn.cursor()


In [196]:

user_agent = "Searchbot_01"
reddit = praw.Reddit(username="Searchbot_01",
                     password ="aaaa1111",
                     client_id="Ai32qfXNqvGuMEvHFFMlAw",
                     client_secret="IG5XKjyUGkcG2cgXfBSwVvalMTxFRg",
                     user_agent=user_agent)

subreddit_name = "watchexchange"
subreddit = reddit.subreddit(subreddit_name)
#print(subreddit.display_name)


df = pd.DataFrame() # creating dataframe for displaying scraped data

# creating lists for storing scraped data
title=[]
upvotes=[]
post_id=[]
time_created = []
#comments = []
url= []
comments = []
#price = ['USD', 'PRICE','$']
upvote_ratio = []

In [197]:
for submission in subreddit.new(limit=10):
    title.append(submission.title)
    upvotes.append(submission.score) #upvotes
    post_id.append(submission.id)
    url.append(submission.url)
    time_created.append(datetime.datetime.utcfromtimestamp(submission.created))
    upvote_ratio.append(submission.upvote_ratio)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [198]:
df['post_id'] = post_id
df['title'] = title
df['upvotes'] = upvotes #upvotes
df['time_created'] = time_created
df['url'] = url
df['upvote_ratio'] = upvote_ratio

In [199]:
#df.set_index('post_id', inplace=True)
print(df.shape)
df.head(10)
print(df)
print(df.columns)

(10, 6)
   post_id                                              title  upvotes  \
0  11jkfdi  [WTS] Forstner flat link bracelet (reserved fo...        1   
1  11jk9pp  [WTS/WTT] Seiko 7548-7000 Quartz Diver with ra...        1   
2  11jk5yl                       [WTS] Rolex Oysterdate 6694!        1   
3  11jk5gi  [WTS] STUNNING Rolex Datejust 16233 w/RARE bla...        2   
4  11jk572  [WTS] Omega Seamaster 300 Ceramic Case w/titan...        2   
5  11jk1ab              [WTS] Wolf Heritage 5 Piece Watch Box        2   
6  11jjw4d  [WTS] Omega Seamaster 300 Chrome Dial Ref:210....        3   
7  11jjvt7                    [WTS/WTT] Seiko 62MAS 6217-8001        2   
8  11jjld5       [WTS] Serge Model 1, Grey Dial, Bronze Bezel        1   
9  11jjlca  [WTS] Timex Weekender and Timex Intelligent Fl...        1   

         time_created                                     url  upvote_ratio  
0 2023-03-06 01:31:59     https://i.redd.it/dzksy75oc2ma1.jpg           1.0  
1 2023-03-06 01:25:18

In [200]:
try:
  df.to_csv('df1.csv')
  print('Success')
except:
  print('Failed')

Success


In [201]:
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS watch_exchange (
                    post_id VARCHAR(10) PRIMARY KEY,
                    title VARCHAR(255),
                    upvotes INTEGER NOT NULL,
                    time_created DATE NOT NULL DEFAULT CURRENT_DATE,
                    url VARCHAR(255),
                    upvote_ratio decimal NOT NULL           
            )""")
    curr.execute(create_table_command)

def insert_into_table(curr, post_id, title, upvotes, time_created, url, upvote_ratio):
  insert_into_exchange = ("""INSERT INTO watch_exchange (post_id, title,  upvotes, time_created, url, upvote_ratio) VALUES (%s,%s,%s,%s,%s,%s);""")
  row_to_insert = (post_id, title,  upvotes, time_created, url, upvote_ratio)
  curr.execute(insert_into_exchange, row_to_insert)

def update_row(curr,title,post_id,upvotes,time_created,url,upvote_ratio):
  query = ("""UPDATE watch_exchange SET title = %s, upvotes = %s, time_created = %s, url= %s, upvote_ratio = %s where post_id = %s;""")
  vars_to_update = (title, upvotes, time_created, url, upvote_ratio, post_id)
  curr.execute(query, vars_to_update)
  
def check_if_post_exists(curr, post_id):
  query = ("""SELECT post_id FROM watch_exchange where post_id = %s""")
  curr.execute(query, (post_id,))
  return curr.fetchone() is not None

def truncate_table(curr):
    truncate_table = ("""TRUNCATE TABLE watch_exchange""")

    curr.execute(truncate_table)
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr,row['post_id'], row['title'],  row['upvotes'], row['time_created'], row['url'],row['upvote_ratio'])
        
def update_db(curr,df):
  tmp_df = pd.DataFrame(columns = ['post_id', 'title', 'upvotes', 'time_created', 'url',  'upvote_ratio'])
  for i, row in df.iterrows():
    if check_if_post_exists(curr, row['post_id']):
      update_row(curr,row['post_id'], row['title'],row['upvotes'],row['time_created'],row['url'], row['upvote_ratio'])
    else:
      tmp_df = tmp_df.append(row)
  return tmp_df

In [202]:
#database credentials
host_name = 'my-db1.cwc1bi2tp4j7.us-east-2.rds.amazonaws.com'
dbname = 'initial_db1'
port = '5432'
username = 'postgres' 
password = 'Manonthemoon123'
conn = None

#establish a connection to db
conn = connect_to_db(host_name, dbname, port, username, password)
curr = conn.cursor()

Connected!


In [203]:
df


,post_id,title,upvotes,time_created,url,upvote_ratio
0,11jkfdi,[WTS] Forstner flat link bracelet (reserved fo...,1,2023-03-06 01:31:59,https://i.redd.it/dzksy75oc2ma1.jpg,1.0
1,11jk9pp,[WTS/WTT] Seiko 7548-7000 Quartz Diver with ra...,1,2023-03-06 01:25:18,https://imgur.com/a/d5VAJOK/,1.0
2,11jk5yl,[WTS] Rolex Oysterdate 6694!,1,2023-03-06 01:20:30,https://i.redd.it/t03gjhyna2ma1.jpg,1.0
3,11jk5gi,[WTS] STUNNING Rolex Datejust 16233 w/RARE bla...,2,2023-03-06 01:20:00,https://www.reddit.com/gallery/11jk5gi,1.0
4,11jk572,[WTS] Omega Seamaster 300 Ceramic Case w/titan...,2,2023-03-06 01:19:41,https://i.redd.it/355rmdd5r0ma1.jpg,1.0
5,11jk1ab,[WTS] Wolf Heritage 5 Piece Watch Box,2,2023-03-06 01:14:52,https://www.reddit.com/gallery/11jk1ab,1.0
6,11jjw4d,[WTS] Omega Seamaster 300 Chrome Dial Ref:210....,3,2023-03-06 01:08:32,https://i.redd.it/l5c72hnsp0ma1.jpg,1.0
7,11jjvt7,[WTS/WTT] Seiko 62MAS 6217-8001,2,2023-03-06 01:08:10,https://imgur.com/a/xzRvva7/,1.0
8,11jjld5,"[WTS] Serge Model 1, Grey Dial, Bronze Bezel",1,2023-03-06 00:55:43,https://v.redd.it/jt3wu5a862ma1,1.0
9,11jjlca,[WTS] Timex Weekender and Timex Intelligent Fl...,1,2023-03-06 00:55:41,https://imgur.com/a/ZZ5hzfN,1.0


In [204]:
create_table(curr)
conn.commit()

In [205]:
new_post_df = update_db(curr,df)
conn.commit()

In [206]:
append_from_df_to_db(curr, new_post_df)
conn.commit()


In [207]:
curr.execute("SELECT * FROM watch_exchange")
print(curr.fetchall())

[('11ji43e', '[WTS] Blancpain Bathyscaphe Ref:50001110B52A', 1, datetime.date(2023, 3, 5), None, Decimal('1.0')), ('11jhw8w', '[WTS] Mr. Jones Perfectly Useless Afternoon', 1, datetime.date(2023, 3, 5), None, Decimal('1.0')), ('11jhlnh', '[WTS] Frederique Constant Limited Edition Rally Healey Challenge Canada 🍁', 1, datetime.date(2023, 3, 5), None, Decimal('1.0')), ('11jhlh9', '[WTS] G-SHOCK GM-2100-1A', 2, datetime.date(2023, 3, 5), None, Decimal('1.0')), ('11jhh8x', '[WTS] Omega Seamaster 300m Quartz 36mm Polished Bezel Insert 2562.80 - Heavy Patina', 2, datetime.date(2023, 3, 5), None, Decimal('1.0')), ('11jhfnj', '[WTS] Lorier Neptune III - Great Condition, Free shipping, $340.', 1, datetime.date(2023, 3, 5), None, Decimal('1.0')), ('11jhdk4', '[WTS] Zodiac GMT Quartz Cushion Case', 1, datetime.date(2023, 3, 5), None, Decimal('1.0')), ('11jhb96', '[WTS] Straton Tourer 43mm Quartz GMT 1-of-1 Blue/Orange Prototype', 1, datetime.date(2023, 3, 5), None, Decimal('1.0')), ('11jhb4v', '[W

In [208]:
#insert_into_table(curr,row['title'],row['post_id'],row['upvote',row['time_created'],row['author'],row['upvote_ratio'])

# New Section